In [1]:
import pandas as pd
import numpy as np
import h5py
import os
import librosa
import argparse
import tqdm

# Arguments

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("-sr", "--sampling_rate", default=22050, type=int, help="Librosa load sampling rate")
parser.add_argument("-l", "--length", default=3, type=int, help="audio file length")
parser.add_argument("-a", "--audioroot", default="../dataset/audio", help="path to audio folds directory")
parser.add_argument("--compression", default=None, type=str)
args = parser.parse_args("")

# Read the metadata

In [3]:
dataset_root = "/corpus/corpus/ESC-50-master/"
metadata = "meta/esc50.csv"
audio = "audio"

metadata_path = os.path.join(dataset_root, metadata)
audio_root = os.path.join(dataset_root, audio)

In [4]:
df = pd.read_csv(metadata_path)

In [12]:
df.loc[df.target == 0]

,fold,target,category,esc10,src_file,take
filename,,,,,,
1-100032-A-0.wav,1,0,dog,True,100032,A
1-110389-A-0.wav,1,0,dog,True,110389,A
1-30226-A-0.wav,1,0,dog,True,30226,A
1-30344-A-0.wav,1,0,dog,True,30344,A
1-32318-A-0.wav,1,0,dog,True,32318,A
1-59513-A-0.wav,1,0,dog,True,59513,A
1-85362-A-0.wav,1,0,dog,True,85362,A
1-97392-A-0.wav,1,0,dog,True,97392,A
2-114280-A-0.wav,2,0,dog,True,114280,A


In [7]:
df = df.set_index("filename")

In [10]:
type(df.at["1-101296-A-19.wav", "esc10"])

numpy.bool_

In [41]:
def load_file(f):
    SR = args.sampling_rate
    LENGTH = args.length
    
    path = os.path.join(audio_root, f)
    raw, sr = librosa.load(path, sr=SR, res_type="kaiser_fast")

    # padding
    if len(raw) < SR * LENGTH:
        missing = (SR * LENGTH) - len(raw)
        raw = np.concatenate((raw, [0] * missing))
        
    elif len(raw) > SR * LENGTH:
        raw = raw[:SR*LENGTH]

    return raw

In [42]:
to_remove = [".DS_Store"]

# Create the HDF file and prepare the 5 folders + the "US" folder

In [66]:
!rm /corpus/corpus/ESC-50-master/esc.hdf

In [67]:
hdf = h5py.File(os.path.join(dataset_root, "esc.hdf"), "w")

In [68]:
df.loc[df.fold == 1].esc10.values

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True, False,  True,  True,  True, False, False, False, False,
        True,  True, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True,  True,  True, False, False, False, False,
       False, False, False, False, False,  True,  True,  True,  True,
       False, False,  True,  True, False,  True,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [69]:
for i in range(5):
    hdf_group = hdf.create_group("fold%s" % (i+1))
    
    # load the important metadata
    filenames = df.loc[df.fold == (i+1)].filename.values
    category = df.loc[df.fold == (i+1)].category.values
    esc10 = df.loc[df.fold == (i+1)].esc10.values
    
    # Extract audio for the corresponding fold
    raw_audios = []
    for name in tqdm.tqdm_notebook(filenames):
        raw_audios.append(load_file(name))
        
    # Store all information and audio into the HDF correspond group
    hdf_group.create_dataset("data", data=raw_audios, compression=args.compression)
    hdf_group.create_dataset("esc10", data=esc10)
    
    hdf_group.create_dataset("category", (len(category), ), dtype=h5py.special_dtype(vlen=str))
    hdf_group.create_dataset("filenames", (len(filenames), ), dtype=h5py.special_dtype(vlen=str))
    for i in range(len(filenames)):
        hdf_group["filenames"][i] = category[i]
        hdf_group["filenames"][i] = filenames[i]

<ipython-input-69-74cc73743b97>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for name in tqdm.tqdm_notebook(filenames):


In [76]:
hdf.close()